In [1]:
import json
import random
from functools import reduce
from math import log

from PIL import Image

In [3]:
# READ IN TRAITS DATA FROM JSON AND CREATE CONSTANTS

# trait data consists of two arrays : [trait_names, trait_rarity_percentage]
TRAIT_WEIGHTS = None 
with open("./output/traitWeights.json", "r") as rf:
    TRAIT_WEIGHTS = json.load(rf)

TRAIT_TYPES = ["Background", "Class", "Body", "Head", "Eyes", "Mouth", "Back"]
# Background and class are the only types that dont rely on class to generate
CLASS_DEPENDENT_TRAIT_TYPES = TRAIT_TYPES[2:]

In [4]:
# utilities
def check_humanoid(class_name): return class_name not in ["Dragon", "Golem", "Phantom"]
def check_phantom(class_name): return class_name == "Phantom"
def get_base_class(class_name): return "Humanoid" if check_humanoid(class_name) else class_name

<center><h1>Asset Path Resolver</h1></center>

In [5]:
# static paths
ASSET_PATH = "../assets/images/"
# use this as a default when none trait
NONE_PATH = "../assets/images/none.png"
# background uses a centralized folder
def get_bg_path(bg_name): return f"{ASSET_PATH}/background/{bg_name.lower()}.png"
# classname only matters for things that are humanoid or phantom
def get_class_path(class_name): return f"{ASSET_PATH}/class/{class_name.lower()}.png" if check_humanoid(class_name) or check_phantom(class_name) else None;
# general utility for getting asset image path
def get_trait_path(traits, trait_type):
    # return none default if trait is "none"
    if traits[trait_type] == "None": return NONE_PATH
    # format trait name to match image file names 
    trait_type_path = trait_type.lower()
    trait_path = traits[trait_type].replace(" ", "_").lower()
    # the class path needs to be resolved
    base_class_path = get_base_class(traits["Class"]).lower()
    return f"{ASSET_PATH}/{base_class_path}/{trait_type_path}/{trait_path}.png"

<center><h1>Generating File Name</h1></center>

In [6]:
def get_trait_index(trait_type, trait_name, base_class_name):
    is_class_dependent = trait_type in CLASS_DEPENDENT_TRAIT_TYPES
    trait__type_info = TRAIT_WEIGHTS[trait_type][base_class_name] if is_class_dependent else TRAIT_WEIGHTS[trait_type]
    trait_index = trait__type_info["traits"].index(trait_name)
    return trait_index

def generate_file_name(traits):
    # for each trait get its index in the json object and pad with one '0'
    base_class_name = get_base_class(traits["Class"])
    trait_idxs = map(lambda trait_type: get_trait_index(trait_type, traits[trait_type], base_class_name), CLASS_DEPENDENT_TRAIT_TYPES)
    # prepend path with power so they sort by power
    path_name = f"{traits['Power']}"
    for idx in trait_idxs:
        path_name = path_name + f"{idx:02d}"
    return path_name

<center><h1>Trait Resolver</h1></center>

In [7]:

def get_random_trait_from_type_info(type_info): return random.choices(type_info["traits"],type_info["weights"])[0]
def get_random_trait(type, class_name = None): 
    # get random background or class
    if(class_name == None): return get_random_trait_from_type_info(TRAIT_WEIGHTS[type])
    # get resolved class name to find trait in trait data
    resolved_class = get_base_class(class_name)
    trait_type_info = TRAIT_WEIGHTS[type][resolved_class]
    # chose first random trait returned
    trait = get_random_trait_from_type_info(trait_type_info)
    return trait

<center><h1>Power Level</h1></center>

In [8]:
# calculate power contribution for a single trait
def calc_power_contribution(traits, trait_type, base_power_weight):
    base_class = get_base_class(traits["Class"])
    trait_name = traits[trait_type]
    is_class_dependent = trait_type in CLASS_DEPENDENT_TRAIT_TYPES
    trait_type_info = TRAIT_WEIGHTS[trait_type][base_class] if is_class_dependent else TRAIT_WEIGHTS[trait_type]
    # retrieve the rarity for the given trait
    # need to find the index of the trait name 
    trait_idx = trait_type_info["traits"].index(trait_name)
    # use idx to index the rarities array
    trait_rarity =  trait_type_info["weights"][trait_idx]
    # dampening factor (use log to make sure numbers remain somewhat closer together at scale)
    # add 1 in order to ensure dampen is never 0
    dampen =  (1 + abs(log(trait_rarity))) / 2
    # dampen the weight that was given for this trait, 
    # the dampening factor will be smaller if the rarity is "high" AKA low percentage => higher power
    return base_power_weight / dampen

In [9]:
def get_power(traits):
    base_class = get_base_class(traits["Class"]);
    is_humanoid = check_humanoid(base_class)
    is_phantom = check_phantom(base_class)
    is_special = not is_humanoid and not is_phantom
    is_golem = base_class == "Golem"
    
    # Create weights for each trait that are multipliers to its power
    # each trait has separate scaling factors based on the class to balance power levels
    BASE_POWER_WEIGHTS = [
        500,                                                        # background
        3000,                                                       # class
        1800 * (1 if not is_special else 2),                        # body
        1200 * (1 if not is_special else 2 if is_golem else 1.5),   # head
        800 * (1 if not is_special else 1.6),                       # eyes
        650,                                                        # mouth
        200 * (1 if not is_special else 2 if is_golem else 1.6)     # back
    ]
    
    # calculate the power contribution for each trait
    trait_powers = [calc_power_contribution(traits, type, bpw) for type, bpw in zip(TRAIT_TYPES, BASE_POWER_WEIGHTS)]
    
    # sum powers and round
    power = round(reduce(lambda curr, tot: curr + tot, trait_powers))
    
    return power
    

<center><h1>Generate Trait Metadata</h1></center>

In [10]:
def generate_metadata(n = 200):
    metadata = []
    # used for checking if metadata already exists easily
    file_names = []
    i = 0
    while i < n:
        # generate background and class first since they are exceptions
        new_background = get_random_trait("Background")
        new_class = get_random_trait("Class")
        # generate rest of traits
        potr_metadata = {
            "Background": new_background,
            "Class": new_class,
            "Body": get_random_trait("Body", new_class),
            "Head": get_random_trait("Head",new_class),
            "Eyes": get_random_trait("Eyes",new_class),
            "Mouth": get_random_trait("Mouth",new_class),
            "Back":  get_random_trait("Back",new_class)
        }
        
        # retrieve the power for these traits
        power = get_power(potr_metadata)
        # add power to traits and add it to list of metadata
        potr_metadata["Power"] = power
        
        # loop again if these traits exist
        file_name = generate_file_name(potr_metadata) 
        if(file_name in file_names):
            continue;
        else:
            i += 1
            file_names.append(file_name)
            metadata.append(potr_metadata)
    
    # return metadata once all of the requested objects have been made
    return metadata

<center><h1>------------- GENERATE POTRS HERE -------------</h1></center>

In [11]:
# n = # of nfts to makew
NUM_POTRS = 6000
potr_traits = generate_metadata(n = NUM_POTRS)

ValueError: The number of weights does not match the population

In [854]:
# reduce function that goes thru all potr traits and tallies each one into an object
def get_trait_count(count, curr_traits): 
    for trait_type in TRAIT_TYPES:
        trait_name = curr_traits[trait_type]
        
        # init count for trait_type
        if(trait_type not in count): count[trait_type] = {}
        
        # init count for trait
        if(trait_name not in count[trait_type]):
            count[trait_type][trait_name] = 1 
            continue;

        # increment
        count[trait_type][trait_name] += 1
    return count

# same thing as above but with power
def get_power_count(count, curr_traits): 
    # get power
    power = curr_traits["Power"]
    
    # init count for power
    if(power not in count): count[power] = 0

    # increment
    count[power]+= 1
    
    return count
        
# calculates the actual rarity for each trait 
def get_trait_true_rarities(stats, trait_type):
    # iterate over trait names in category
    for trait_name, count in stats[trait_type].items():
        # calculate the rarity fraction of trait
        true_rarity = round(count * 100 / NUM_POTRS, 3)
        # add true rarity to stats
        stats[trait_type][trait_name] = [count, true_rarity]
    return stats

In [866]:
trait_counts = reduce(get_trait_count, potr_traits, {})
trait_true_rarities = reduce(get_trait_true_rarities, TRAIT_TYPES, trait_counts)

# sort by descending power if needed
potr_traits.sort(reverse=True, key=(lambda traits: traits["Power"]))

power_counts = reduce(get_power_count, potr_traits, {})

most_frequent_power = reduce(lambda max, power: [power, power_counts[power]] if power_counts[power] > max[1] else max, power_counts, [0,0])
highest_power = reduce(lambda max, power: power if power > max else max, power_counts)
print(f"most_frequent_power: {most_frequent_power}")
print(f"highest_power: {highest_power}")

# write traits to json if needed
with open('./output/potrTraits.json', 'w') as f:
    json.dump(potr_traits, f, indent=2)
    
with open('./output/traitTrueRarities.json', 'w') as f:
    json.dump(trait_true_rarities, f, indent=2)

[4674, 12]
10138


In [865]:
filenames = [generate_file_name(traits) for traits in potr_traits]
print(f"number of potrs: {len(potr_traits)}")
print(f"number of filenames generated: {len(filenames)}")
print(f"number of unique filenames: {len(set(filenames))}")

6000
6000
6000


<center><h1>Generate Images</h1></center>

In [860]:
def generate_layers(traits):
    layers = {
        "Background": Image.open(get_bg_path(traits["Background"])).convert('RGBA'),
        "Body": Image.open(get_trait_path(traits, "Body")).convert('RGBA'),
        "Head": Image.open(get_trait_path(traits, "Head")).convert('RGBA'),
        "Eyes": Image.open(get_trait_path(traits, "Eyes")).convert('RGBA'),
        "Mouth": Image.open(get_trait_path(traits, "Mouth")).convert('RGBA'),
        "Back": Image.open(get_trait_path(traits, "Back")).convert('RGBA'),
    }
    return layers
def generate_dragon_layers(traits):
    # dragons back assets have name based on color which does not match the trait name
    back_trait = traits["Body"] if traits["Back"] == "Dragon Wings" else traits["Back"]
    traits["Back"] = back_trait
    layers = generate_layers(traits)   
    # dragons body covers as its class
    layers["Class"] = Image.open(NONE_PATH).convert('RGBA')
    # put the correct back name back into traits
    if(traits["Back"] not in TRAIT_WEIGHTS["Back"]["Dragon"][0]): traits["Back"] = "Dragon Wings"
    return layers
def generate_golem_layers(traits):
    layers = generate_layers(traits)   
    layers["Class"] = Image.open(NONE_PATH).convert('RGBA')
    return layers
def generate_humanoid_layers(traits):
    layers = generate_layers(traits)
    layers["Class"] = Image.open(get_class_path(traits["Class"])).convert('RGBA')
    return layers

In [861]:
#  this takes the layers and creates an actual image
def create_image_composite(traits, layers):
    trait_types = TRAIT_TYPES.copy()
    
    # if not humanoid remove class so we dont add an unnecessary layer for class trait
    if not check_humanoid(traits["Class"]): trait_types.remove("Class")
    
    # create first layer
    potr = Image.alpha_composite(layers["Background"], layers["Back"]);
    
    # remove used layers
    trait_types.remove("Background")
    trait_types.remove("Back")
    
    # add rest of layers
    for type in trait_types:
        potr = Image.alpha_composite(potr, layers[type])
        
    # Convert to RGB so there is no opacity aspect
    potr = potr.convert('RGB')
    return potr
            

<center><h1>------------- GENERATE IMAGES HERE -------------</h1></center>

In [867]:
for traits in potr_traits:
    class_name = traits["Class"]
    
    layers = None;
    if(class_name == "Dragon"):
        layers = generate_dragon_layers(traits)
    elif(class_name == "Golem"):
        layers = generate_golem_layers(traits)
    else:
        # generating humanoid will be same as phantom
        layers = generate_humanoid_layers(traits)

    # create composite images
    potr = create_image_composite(traits, layers)

    # generate file name based on traits
    potr_name = generate_file_name(traits)
    
    # save the new image file
    potr.save(f"../nfts/{potr_name}.png")